# PCA tidyverse style
This file reads in a csv file from the "data" folder and creates PCA plots of selected species.

## 1. Setup

Install the necessary packages. You can check whether you have these packages already installed by running `conda list`.

In [ ]:
install.packages("readr")
install.packages("dlookr")
install.packages("devtools")
install.packages("tidyverse")
install.packages("ggfortify")
install.packages("cowplot")
install.packages("plotly")
install.packages("ggrepel")
devtools::install_github("tidymodels/broom")

If any packages fail to install with the cell block above, try to install using the following commands
- `conda install r-devtools r-tidyverse`
- `conda install conda-forge::r-dlookr conda-forge::r-ggfortify conda-forge::r-cowplot conda-forge::r-plotly conda-forge::r-ggrepel`

In [ ]:
# import libraries
library(scales)
library(readr)
library(dlookr)
library(dplyr)
library(broom)
library(ggplot2)
library(cowplot)
library(plotly)
library(htmltools)
library(tidyr)
library(ggrepel)

## 2. Customization

Change the values in this section to adapt this program to your species. Then, run all code cells below this section to generate your PCA plots. There is no need to modify code cells outside of this section.

In [ ]:
# Change filename to match the name and directory of your data file. Datafile should be complete (no missing values)
input_file = "data/morph.csv"

In [ ]:
# Filename for the eigenvalue barplot that will be generated. Change name as desired.
ev_file = "output/eigenvalue barplot.pdf"

# Filename for pca plots that will be generated. Change names as desired.
pca_static_file = "output/static pca plot.pdf"
# Note: saving the dynamic pca plot in the same folder as this .ipynb file prevents creation of an extra folder for html components
pca_dynamic_file = "dynamic pca plot.html"

# Filename for csv of PCA variable coordinates that will be generated. Change name as desired.
pca_var_file = "output/pca var.csv"

# Filename for rotation matrix plot that will be generated. Change name as desired.
rotation_file = "output/rotation matrix.pdf"

In [ ]:
# List of species to include in the PCA plot. Change as needed
species_list = c("E. jamesdixoni", "E. nitidus")

In [ ]:
# custom qualitative color palettes
palette_colorblind <- c("#E69F00",
                        "#56B4E9",
                        "#009E73",
                        "#F0E442",
                        "#0072B2",
                        "#D55E00",
                        "#CC79A7",
                        "#999999")
palette_cb_ext <- c("#ebac23",
                    "#b80058",
                    "#008cf9",
                    "#006e00",
                    "#00bbad",
                    "#d163e6",
                    "#b24502",
                    "#ff9287",
                    "#5954d6",
                    "#00c6f8",
                    "#878500",
                    "#00a76c",
                    "#979797",
                    "#1e1e1e")

print("Color palettes:")
par(mfrow=c(1,2))            # output color previews on 2 lines
show_col(palette_colorblind) # colorblind-friendly palette
show_col(palette_cb_ext)     # extended colorblind-friendly palette

## 3. Data Preprocessing

In [ ]:
# Read in csv file as tibble
morph <- read_csv(input_file, col_types = cols(Specimen = "c", Species = "f"))
# keep only the data for species in species_list
morph <- morph %>% filter(Species %in% species_list)
morph

In [ ]:
# Check for skewness with dlookr package
# find names of skewed variables
find_skewness(morph, index = FALSE)

# compute the skewness
find_skewness(morph, value = TRUE)

In [ ]:
# log10 transform continuous variables
log10.morph <- morph %>% 
    transmute(
        ID = Specimen,
        Species = Species,
        log.HW = log(HW),
        log.SVL = log(SVL),
        log.IOD = log(IOD),
        log.ED = log(ED),
        log.IND = log(IND),
        log.EN = log(EN),
        log.FL = log(FL),
        log.TD = log(TD),
        log.THL = log(THL),
        log.HAL = log(HAL),
        log.FLL = log(FLL),
        log.Fin3L = log(Fin3L),
        log.Fin3DW = log(Fin3DW),
        log.Fin4L = log(Fin4L),
        log.Fin4DW = log(Fin4DW)
    )

## 4. PCA

### a) Perform PCA

In [ ]:
## Center and scale data to unit variance and perform PCA ##
pca_fit <- log10.morph %>%
  select(where(is.numeric)) %>% 
  prcomp(center = TRUE,
         scale = TRUE)

Combine PC coordinates with the original dataset to color samples based on the categorical variable (Species) which had been removed during the PCA. 
This is done using the augment() function from the broom package, which combines the fitted model with the original data. 
Columns with fitted coordinates are named .fittedPC1, .fittedPC2, etc.

In [ ]:
# assign fitted model plus original data to new variable pca_fit2
pca_fit2 <-
  pca_fit %>% 
  augment(log10.morph)  

pca_fit2

### b) Visualize the variance explained by each PC

In [ ]:
pca_fit %>%
  tidy(matrix = "eigenvalues")

In [ ]:
# Make barplot of PCs
x <- pca_fit %>%
  tidy(matrix = "eigenvalues") %>%
  ggplot(aes(PC, percent)) +
  xlab ("PC") + ylab ("Percent Variance") +
  geom_col(alpha = 0.8) +
  scale_x_continuous(breaks = 1:9) +
  scale_y_continuous(labels = scales::percent_format(),
                     expand = expansion(mult = c(0, 0.01))) +
  theme_minimal_hgrid(12)

barplot = plot(x)

In [ ]:
# save barplot to a pdf file named "PCA barplot.pdf"
ggsave(filename=ev_file, plot=barplot, device="pdf")

### c) Plot each sample using the first two PCs.

In [ ]:
# Plot biplot colored by species to look for separation between groups
p <- ggplot(pca_fit2,
            aes(x = .fittedPC1,
                y = .fittedPC2,
                text = paste("Specimen:", ID))) + 
     geom_point(color = 'black',
                shape = 21,
                size = 3,
                aes(fill = Species)) +
     scale_fill_manual(values = palette_cb_ext) +
     theme_half_open(12) + 
     background_grid()

In [ ]:
# create static plot
static_pca <- plot(p)

In [ ]:
# save static plot as pdf
ggsave(filename=pca_static_file, plot=static_pca, device="pdf")

In [ ]:
# use ggplotly to convert ggplot2 object to plotly object for interactive plot
interactive_plot <- ggplotly(p)  

# display interactive plot in new tab
html_print(
  interactive_plot,
  viewer = getOption("viewer", utils::browseURL)
)

In [ ]:
# save interactive plot for future reference
htmlwidgets::saveWidget(as_widget(interactive_plot), pca_dynamic_file)

### d) Visualize rotation matrix

In [ ]:
# Extract rotation matrix
pca_fit %>%
  tidy(matrix = "rotation")

In [ ]:
# Examine factor loadings from rotation matrix
PCA_var <- pca_fit %>%
  # Extract variable coordinates
  tidy(matrix = "rotation") %>%
  # Format table form long to wide
  pivot_wider(names_from = "PC",
              names_prefix = "PC",
              values_from = "value") %>%
  # Rename column with variable names; 
  # specify dplyr to avoid package conflicts with 'rename'
  dplyr::rename(Variable = column) %>%
  # 'Clean' variable names
  # Upper case on first letter
  mutate(Variable = stringr::str_to_title(Variable)) %>%
  # Change '_' for space
  mutate(Variable = stringr::str_replace_all(Variable, "_", " "))

PCA_var

In [ ]:
# save PCA_var to csv file
write_csv(PCA_var, pca_var_file)

In [ ]:
# calculate cutoff for 'important' loadings
cutoff <- morph %>% select(!c(Specimen, Species))
sqrt(1/ncol(cutoff))

In [ ]:
# Define arrow style for plotting
arrow_style <- arrow(
    angle = 20,
    ends = "first",
    type = "closed",
    length = grid::unit(8, "pt")
)

In [ ]:
# Plot rotation matrix; use geom_text_repel to avoid label overlap 
rotation_matrix <- pca_fit %>% 
    tidy(matrix = "rotation") %>%
    pivot_wider(names_from = "PC",
                names_prefix = "PC",
                values_from = "value") %>%
    ggplot(aes(PC1, PC2)) +
    geom_segment(xend = 0, yend = 0, arrow = arrow_style) +
    geom_text_repel(aes(label = column),
                    hjust = 1,
                    nudge_x = -0.02,
                    color = "#904C2F") +
    xlim(-0.5, 0.5) + 
    ylim(-0.5, 0.5) +
    coord_fixed() + # fix aspect ratio to 1:1
    theme_minimal()

rotation_matrix

In [ ]:
# save rotation matrix plot to pdf
ggsave(filename=rotation_file, plot=rotation_matrix, device="pdf")

In [ ]:
# Display session info
devtools::session_info()